# Initialisation de Spark

In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Notebook").setMaster("local")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/21 18:49:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/21 18:49:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# Une fonction pour simplifier l'accès aux données

In [2]:
def extract_field(s: str, field_number: int, sep: str = ';') -> str:
    fields = s.split(sep)
    return fields[field_number] if (field_number < len(fields)) else ""

In [3]:
print(extract_field("2;CASSIOPEE;2009;33;3", 0))
print(extract_field("2;CASSIOPEE;2009;33;3", 1))
print(extract_field("2;CASSIOPEE;2009;33;3", 2))
print(extract_field("2;CASSIOPEE;2009;33;3", 3))
print(extract_field("2;CASSIOPEE;2009;33;3", 4))
print(extract_field("2;CASSIOPEE;2009;33;3", 5))

2
CASSIOPEE
2009
33
3



# Charger les données
1. Créer le RDD `lignes` à partir du répertoire `prenoms_sample.txt`

In [4]:
lignes = sc.textFile("prenoms_sample.txt")
print(lignes.take(10))

['2;MARIE-JOSÉ;1955;30;21', '2;MARIE-JOSÉ;1956;56;9', '2;MARIE-JOSÉ;1966;64;29', '2;MARIE-JOSEE;1951;59;22', '2;MARIE-JOSEE;1954;54;6', '2;MARIE-JOSEE;1954;67;15', '2;MARIE-JOSEPH;1907;56;4', '2;MARIE-JOSEPHE;1942;75;11', '2;MARIE-JOSEPHE;1951;78;3', '2;MARIE-LAURE;1958;61;6']


# Transformer les lignes en prénoms
1. En appliquant la méthode `map`, créer le RDD `prenoms` à partir de `lignes`

In [5]:
prenoms = lignes.map(lambda l: (
    extract_field(l, 0)[0],
    extract_field(l, 1),
    int(extract_field(l, 2)),
    extract_field(l, 3),
    int(extract_field(l, 4))
))
for n in prenoms.take(10):
    print(n)

('2', 'MARIE-JOSÉ', 1955, '30', 21)
('2', 'MARIE-JOSÉ', 1956, '56', 9)
('2', 'MARIE-JOSÉ', 1966, '64', 29)
('2', 'MARIE-JOSEE', 1951, '59', 22)
('2', 'MARIE-JOSEE', 1954, '54', 6)
('2', 'MARIE-JOSEE', 1954, '67', 15)
('2', 'MARIE-JOSEPH', 1907, '56', 4)
('2', 'MARIE-JOSEPHE', 1942, '75', 11)
('2', 'MARIE-JOSEPHE', 1951, '78', 3)
('2', 'MARIE-LAURE', 1958, '61', 6)


# Interroger les données
La documentation des méthodes d'un RDD est disponible ([RDD](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.html)).

1. Rappeler ce que sont les *transformations* et les *actions*

En Spark, les *transformations* décrivent les opérations à faire sur les données sans les exécuter, tandis que les *actions* déclenchent réellement leur exécution pour produire un résultat.

2. Donner, pour chaque prénom, son nombre d'occurences (`map` et `reduceByKey`)

In [6]:
occ_prenoms = prenoms.map(lambda prenom: (prenom[1], 1)).reduceByKey(lambda a, b: a + b)

for n in occ_prenoms.take(10):
    print(n)

('MARIE-JOSÉ', 3)
('MARIE-JOSEPH', 1)
('MARIE-JOSEPHE', 2)
('MARIE-LAURE', 2)
('MARIE-NADEGE', 1)
('MARIE-NOËLLE', 3)
('MARIE-THÉRÈSE', 9)
('MARIELLE', 1)
('MARIETTE', 2)
('MARILENE', 1)


1. Donner le nombre total de naissances avec un prénom féminin (`filter`, `map`, `reduce` ou `sum`)

In [7]:
nb_naissance_feminin = prenoms.filter(lambda sexe: sexe[0] == '2').map(lambda prenom: prenom[4]).sum()
print(nb_naissance_feminin)

41548


1. Donner l'effectif maximal et minimal par prénom (`map`, `aggregateByKey`)

In [8]:
max_min_prenoms = prenoms.map(lambda prenom: (prenom[1], prenom[4])).aggregateByKey(
    (float('inf'), float('-inf')),
    lambda acc, v: (min(acc[0], v), max(acc[1], v)),
    lambda acc1, acc2: (min(acc1[0], acc2[0]), max(acc1[1], acc2[1]))
    )

for n in max_min_prenoms.take(10):
    print(n)

('MARIE-JOSÉ', (9, 29))
('MARIE-JOSEPH', (4, 4))
('MARIE-JOSEPHE', (3, 11))
('MARIE-LAURE', (5, 6))
('MARIE-NADEGE', (5, 5))
('MARIE-NOËLLE', (7, 12))
('MARIE-THÉRÈSE', (3, 20))
('MARIELLE', (5, 5))
('MARIETTE', (9, 28))
('MARILENE', (3, 3))


1. Sur le modèle des prénoms, charger les données des départements
1. Donner, pour chaque nom de département, le prénom le plus fréquent depuis l'année 2000

In [9]:
dpts = sc.textFile("dpts.txt")
dpts.filter(lambda l: l.startswith("dep") == False)\
    .saveAsTextFile("dpts_sample.txt")


In [14]:
lignes = sc.textFile("dpts_sample.txt")

dpts = lignes.map(lambda l: (
    extract_field(l, 0, ','),
    int(extract_field(l, 1, ',')),
    (extract_field(l, 2, ',')),
    int(extract_field(l, 3, ',')),
    extract_field(l, 4, ','),
    extract_field(l, 5, ','),
    extract_field(l, 6, ',')
))
for n in dpts.take(10):
    print(n)

('01', 84, '01053', 5, 'AIN', 'Ain', 'Ain')
('02', 32, '02408', 5, 'AISNE', 'Aisne', 'Aisne')
('03', 84, '03190', 5, 'ALLIER', 'Allier', 'Allier')
('04', 93, '04070', 4, 'ALPES DE HAUTE PROVENCE', 'Alpes-de-Haute-Provence', 'Alpes-de-Haute-Provence')
('05', 93, '05061', 4, 'HAUTES ALPES', 'Hautes-Alpes', 'Hautes-Alpes')
('06', 93, '06088', 4, 'ALPES MARITIMES', 'Alpes-Maritimes', 'Alpes-Maritimes')
('07', 84, '07186', 5, 'ARDECHE', 'Ardèche', 'Ardèche')
('08', 44, '08105', 4, 'ARDENNES', 'Ardennes', 'Ardennes')
('09', 76, '09122', 5, 'ARIEGE', 'Ariège', 'Ariège')
('10', 44, '10387', 5, 'AUBE', 'Aube', 'Aube')


In [ ]:
freq_depts_prenoms = prenoms.filter(lambda prenom: prenom[2] >= 2000)\
    .map(lambda prenom: ((prenom[1], prenom[3]), prenom[4]))\
    .reduceByKey(lambda a, b: a + b)
freq_by_dept = freq_depts_prenoms.map(lambda x: (x[0][1], (x[0][0], x[1])))  # clé = code_dept, valeur = (prenom, nb_naissances)

dpts_noms = dpts.map(lambda dpt: (dpt[0], dpt[4]))  # clé = code_dept, valeur = nom_dept

jointure = freq_by_dept.join(dpts_noms)\
    .reduceByKey(lambda a, b: a if a[0][1] > b[0][1] else b)\
    .map(lambda x: (x[1][1], x[1][0][0]))

for n in jointure.take(10):
   print(n)

('VAR', 'MANON')
('MORBIHAN', 'JEANNE')
('SEINE ET MARNE', 'EMMA')
('HAUTE SAONE', 'LILOU')
('LOIRE ATLANTIQUE', 'HUGO')
('MEURTHE ET MOSELLE', 'LIAM')
('OISE', 'ESTEBAN')
('AUBE', '_PRENOMS_RARES')
('VOSGES', 'COLINE')
('VIENNE', '_PRENOMS_RARES')
